# ECON 490: Panel Data Regressions (14)

## Prerequisites

1. Run OLS Regressions.

## Learning Outcomes

1. Prepare data for time series analysis. 
2. Run panel data regressions.
3. Conduct post regression tests for panel data regressions.
4. Correct for heteroskedasticity and serial correlation.

This module is undertaken using the [Penn World Tables](https://www.rug.nl/ggdc/productivity/pwt/?lang=en) which measures income, input, output and productivity, covering 183 countries between 1950 and 2019. Before beginning this module, you should download this data in Stata format.

In [ ]:
use pwt100, clear

In [ ]:
describe

## 17.1 What is Panel Data?

The typical structure we study in econometrics consists of many units observed in a particular point in time (henceforth referred as cross-sectional data). However, it is often the case that we get a chance to observe the same unit over many time periods. This data structure is known as panel data. Let us introduce some new definitions:

- Balanced Panel: It is such that we observe *all* units over *all* time periods in the data set. For example, the [Penn World Tables](https://www.rug.nl/ggdc/productivity/pwt/?lang=en) show all countries on every time period. 

- Unbalanced Panel: It is such that we do not necessarily observe all units over all time periods. A data set which records all employment in a country every year, but in which some years some workers are not observed because they have become unemployed, would be an example of this type.

To indicate to Stata that we are working with a panel, we need to create two *numerical* (this is essential) codes that identify the unit and time. In this case, _year_ is already a numerical variable, so we are left to create a numeric country code based on the name of the country. We can run the following command to do this:

In [ ]:
encode country, gen(code) 

The command `xtset` defines the panel variables and provides some options if we want to indicate the periodicity of the observations. 

In [ ]:
xtset code year, yearly

The command states that we observe `codes` (i.e. countries) over many time period years that change over 1 unit (i.e. year after year).

## 17.2 How can It be Helpful?

In typical cross-sectional settings (i.e. where the variables are all measured at a single point in time) it is hard to defend a **selection on observables** assumption. However, panel data allows us to control for unobserved time invariant heterogeneity. 

Consider the following example. Household income  $y_{jt}$ at time $t$ can be split into two components: 

$$
y_{jt} = e_{jt} + \Psi_{j}
$$
    
where $\Psi_{j}$ is a measure of unobserved household-level determinants of income such as social programs targeted towards certain households. 
    
Consider what happens when we compute the average for every household in the data:

$$
\bar{y}_{J}= \frac{1}{\sum_{j,t}   \mathbf{1}\{ j = J \}  } \sum_{j,t}  y_{jt} \mathbf{1}\{ j = J \}
$$
$$
\bar{e}_{J}= \frac{1}{\sum_{j,t}   \mathbf{1}\{ j = J \}  } \sum_{j,t}  e_{jt} \mathbf{1}\{ j = J \}
$$
$$
\bar{\Psi}_{J} =  \Psi_{J}
$$
    
Notice that the mean of $\Psi_{j}$ does not change over time for a fixed household $j$. Hence, we can subtract the household level means from the original equation to get: 
   
$$
y_{jt} - \bar{y}_{j} = e_{jt} - \bar{e}_{j}  + \underbrace{ \Psi_{j} - \bar{\Psi}_{j}  }_\text{equals zero!}
$$
 
Therefore, we got rid of the unobserved heterogeneity via "de-meaning"! If we believe these types of unobserved errors/shocks are creating endogeneity, we can get rid of them using this powerful trick.

## 17.3 Fixed Effects 

We refer to shocks that are invariant based on some variable (e.g. household level shocks that are invariant based on year) as *Fixed Effects*. For instance, we can define household fixed effects, time fixed effects (time shocks that affect all units the same way), and so on. Notice that this is an assumption on the error terms, and as such it becomes part of the model we assume to be true. A well-known result is that controlling for fixed effects is *equivalent* to adding multiple dummy variables. For example, consider a regression of GDP and population where we want to control for country-level shocks that do not vary over time. 

- Approach 1: create country dummies and include them in the regression. The problem with this approach is that we end up with a huge table containing the coefficients of every country dummy, which we don't care about by definition. We are here for the relationship between GDP and population, not the control variables.  

In [ ]:
reg rgdpo pop i.code

- Approach 2: Since we know this is equivalent to demeaning the data based on _code_, there is a command that does this trick. It refers to de-meaning as "absorbing" indicators. Consider:

In [ ]:
areg rgdpo pop, absorb(code)

We obtained the same coefficient and standard errors on *pop* using both approaches!

## 17.4 What if We Want to Control for Multiple Fixed Effects? 

Suppose we want to control for time fixed effects and country fixed effects. We can do so by including dummies for every fixed effect. Let's do that below.

In [ ]:
reg rgdpo pop i.year i.code 

In contrast, _areg_ only allows for one variable to be absorbed. We generally choose the one with more categories to be included.

In [ ]:
areg rgdpo pop i.year , absorb(code)

We see here that the logic of demeaning prevails. However, when we include multiple fixed effects, the de-meaning process works on one fixed effect after the other. That is, we first take country averages, de-mean, and then take time averages to be de-meaned.

## 17.5 Time Series Variables

Panel data has provided a new source of variation: variation over time. This means that we now have access to a wider variety of independent variables. For instance, we could regress population $y_{jt}$ against the population size in the prior year $y_{jt-1}. Once we have defined a panel using `xtset` we can create lags using `L.variable` and leads using `F.variable`.

In [ ]:
reg pop L.pop

We can also control for population in year $t-2$ using `L2.variable`, and so on.

In [ ]:
reg pop L.pop L2.pop

## 17.6 Granger Causality 

When we have variables observed over time, a very natural (and primitive) notion of causality is that if $X$ happens before $Y$, it must've caused it. We refer to this phenomenon as Granger Causality. To test this, we must estimate a regression and test whether the coefficient on lagged variables (e.g. $X_{t-1}$ , $X_{t-2}$, etc.) are significant to predict $Y_t$. Hence we are essentially testing whether lagged variables help us predict another variable of interest.

This notion is related to a time-series variable that does not have any cross-sectional variation. To use these tools as an example, we will keep only the observations of one country.

In [ ]:
keep if country=="Mexico"

Intuitively, what we do is run the regression model

$$
Y_t = \beta_0 + \beta_1 Y_{t-1} + \beta_2 Y_{t-2}  + \gamma_1 X_{t-1} + \gamma_2 X_{t-2} + \epsilon_t
$$

and test whether all the $Y$s are jointly significant or not using an F-test.

In [ ]:
reg rgdpo L.rgdpo L2.rgdpo L.pop L2.pop

In [ ]:
test L.pop L2.pop

We cannot reject the null hypothesis that these two coefficients are zero at the 95% level. Therefore, we have no evidence that population is a Granger cause of GDP.

## 17.7 Dealing with Standard Errors

In time-series setups where we only observe a single unit over time (no cross-sectional dimension) we might be worried that a linear regression model like
$$ Y_t = \alpha + \beta X_t + \varepsilon_t $$

can have errors that not only are heteroskedastic (i.e. that depend on observables $X_t$) but they can also be correlated across time periods. For instance, if this was an equation where $Y_t$ is income then $\varepsilon_t$ represent income shocks (including transitory and permanent components). The permanent income shocks are, by definition, very persistent over time. This means that $ \varepsilon_{t-1}$ affects -and hence is correlated- next period shocks $ \varepsilon_t$. 

These corrected standard errors are called Newey-West Heteroskedastic and Autocorrelated Consisntent (HAC). The command to do so is `newey`. We need to provide an extra number in the Stata command to compute these standard errors, and it is usually the integer approximation to $T^(1/4)$ where $T$ is the number of time periods. 

In [ ]:
di _N^(1/4)

In [ ]:
newey rgdpo L.rgdpo L2.rgdpo L.pop L2.pop, lags(3)

When we are dealing with both a cross-sectional and time dimensions (proper panel), we can allow for shocks at the unit level (e.g. worker) to be correlated over time. It is very simple to implement in Stata. Suppose we want to adjust the following regression  

In [ ]:
use pwt100, clear
encode country, gen(code) 
xtset code year

areg rgdpo pop , absorb(code) robust

We simply change the `robust` option by a `cluster` at the level where we think serial correlation might impact. In this case, if we think that errors within a country are correlated, we write


In [ ]:
areg rgdpo pop , absorb(code) vce(cluster code)

## 17.8 Wrap Up

In this module we've learned how to address linear regression in the case where we have access to two dimensions: cross-sectional and time variation. The usefulness of time variation is that it allows us to control for time-invariant components of the error term that can be causing endogeneity. 

Furthermore, we need to address time variation in the standard errors. When we are working with pure time-series we can construct HAC standard errors, and when we have panel data we can cluster at the unit level. In the next module, we will cover another research design method: difference-in-differences.

## References

[Formatting and managing dates](https://www.youtube.com/watch?v=SOQvXICIRNY&t=149s) <br>
[Time-series operators (lags)](https://www.youtube.com/watch?v=ik8r4WvrPkc&t=224s)